# Homework Assiagment 1 - NLP

## Dor Cohen 302577820

### Part 1: 

Write a function lm that generates a language model	over characters from a textual corpus.	

In [1]:
## here we import staff
from itertools import groupby
from numpy import log2

In [2]:
#### Helpers methods
def n_grams_characters(text, n):
    return [text[i:i+n] for i in range(len(text)-n+1)]

def get_ngrams_dic(grams):
    dic = {}
    for g in grams:
        if g in dic:
            dic[g] = dic[g] + 1
        else:
            dic[g] = 1
    return dic

def write_values_to_file(f, grams_dic, n, prefix_grams_dic={}, total = 0):
    if (n==1):
        f.write("<unk>" +'\t' + str(log2(1/(len(grams_dic) + total))) + '\n') #bounus 
        for k,v in grams_dic.items():
            f.write(str(k) +'\t' + str(log2((v+1)/(total + len(grams_dic)))) + '\n')
        return
    for k,v in grams_dic.items():
        f.write(str(k) +'\t'+ str(log2(v/prefix_grams_dic[k[0:n-1]])) + '\n')
    
        


In [3]:
#The requsted method
def lm(corpus_file, model_file):
    with open(corpus_file, 'r') as f:
        corpus = f.read().lower()
        grams3 = n_grams_characters(corpus, 3)
        grams2 = n_grams_characters(corpus, 2)
        grams1 = n_grams_characters(corpus, 1)
        grams1Dic = get_ngrams_dic(grams1)
        grams2Dic = get_ngrams_dic(grams2)
        grams3Dic = get_ngrams_dic(grams3)
    with open(model_file,'w+') as f:
        f.write('3-grams:\n')
        write_values_to_file(f, grams3Dic,  3, prefix_grams_dic=grams2Dic)
        f.write('2-grams:\n')
        write_values_to_file(f, grams2Dic, 2, prefix_grams_dic=grams1Dic )
        f.write('1-grams:\n')
        write_values_to_file(f, grams1Dic, 1, total = len(corpus))
    
    

### Part 2:
Write	a function eval that calculates and	prints the perplexity of a model running over a	
given text.	


In [5]:
def get_dics_from_model_file(f):
    dict3 = {}
    dict2 = {}
    dict1 = {}
    lines = f.readlines()
    backslash_n_flag = False
    i = 3
    for line in lines:
        line = line.rstrip('\n')
        if line == '3-grams:':
            i = 3
            continue
        elif line == '2-grams:':
            i = 2
            continue
        elif line == '1-grams:':
            i = 1
            continue
        if backslash_n_flag:
            line = '\n' + line
            backslash_n_flag = False
        if line.find('\t')==-1:
            backslash_n_flag = True
            continue
        
        line = line.split('\t')
        if (i==3):
            dict3[line[0]] = float(line[1])
        elif (i==2):
            dict2[line[0]] = float(line[1])
        elif (i==1):
            dict1[line[0]] = float(line[1])
    return [dict3, dict2 , dict1]

In [6]:
def eval(input_file, model_file, weights):
    with open(model_file, 'r') as f:
        dict3, dict2, dict1 = get_dics_from_model_file(f)
    with open(input_file, 'r') as f:
        text = f.read().lower()
        grams = n_grams_characters(text, 3)
        interpolation_dic = {}
        lambda1, lambda2, lambda3 = weights
        sum1 = 0
        for gram in grams:
            P = 0
            if (gram[2:3] in dict1):
                P += lambda3*2**(dict1[gram[2:3]])
            else:
                P += lambda3*2**(dict1['<unk>']) #bonus
            if (gram[1:3] in dict2):
                P += lambda2*2**(dict2[gram[1:3]])
            if (gram in dict3):
                P += lambda1*2**(dict3[gram])
            sum1 += log2(P)
        return 2**((-sum1/len(text)))
       
            
            
            
        

### Part 3:

In [7]:
import pandas as pd
import numpy as np


I will use escape characteres that will indicate EOL and BOL. EOL = '\a' and BOL = '\b'.

In [8]:
languages = ["en", 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
languages_dict = {}
for lan in languages:
    data = pd.read_csv(lan + '.csv')
    data = data.drop("tweet_id", axis=1)
    data['tweet_text'] = '\b' + data['tweet_text'].astype(str) + '\a'
    train = data.sample(frac=0.9,random_state=50)
    test = data.drop(train.index)
    languages_dict[lan] = [train, test]

In [9]:
def add_to_file(x,f):
    f.write(x['tweet_text'])
for lan in languages:
    input_file_name = lan + '_train.txt'
    model_file_name = lan + '_model.txt'
    with open(input_file_name, 'w+') as f:
        train = languages_dict[lan][0]
        train.apply(lambda x: add_to_file(x,f), axis=1)
    lm(input_file_name, model_file_name)

In [10]:
table = np.empty((len(languages)+1, len(languages)+1), dtype=object)
table[0,1:len(languages)+1] = languages
table[1:len(languages)+1,0] = languages
for i in range(1, len(languages)+1):
    for j in range(1, len(languages)+1):
        model_file_name = languages[i-1] + '_model.txt'
        input_file_name = languages[j-1] + '_test.txt'
        with open(input_file_name, 'w+') as f:
            test = languages_dict[languages[j-1]][1]
            test.apply(lambda x: add_to_file(x,f), axis=1)
        result = eval(input_file_name,model_file_name, [0.4, 0.3, 0.3])
        table[i,j] = str(result)


In [11]:
results = pd.DataFrame (table)
results

,0,1,2,3,4,5,6,7,8
0,None,en,es,fr,in,it,nl,pt,tl
1,en,12.076535229087849,17.994538689422196,18.55557944197081,18.539363028422635,17.052876790133404,16.701247898511934,19.748179201333276,17.33174171993408
2,es,17.57630481796407,11.49330046774936,17.417151472296506,19.41609105716152,15.20595659665287,18.425598320833235,15.454487798509813,19.0554785893154
3,fr,16.651978181563926,16.62550893891067,11.824844741116152,19.696575869914657,15.897706339073602,17.49529282958224,17.864357523914652,19.64097804287586
4,in,17.104556657238035,19.41460067477763,21.795873060736664,12.666240406715083,17.919444583945772,18.08236675953222,21.040912553984587,15.907320810732944
5,it,17.30483456028146,15.481373460361235,17.470436828329948,19.536559609238644,11.468008642978926,18.71583919074696,16.524258037845925,17.98548138095934
6,nl,16.221777980937997,18.699089497023447,18.389487732941827,18.722416226615508,18.271995272683895,12.371159453262619,20.063652347231066,18.27024174892468
7,pt,17.785133084964095,14.400730688176896,17.80350295442515,19.872894489006402,15.871853912506372,19.100671273999616,11.334996708770838,18.78590479935641
8,tl,15.979584895763255,17.832543583289866,21.21780830649941,16.09128287286364,17.268251707260934,18.331957869293,19.641922538006423,11.793184868148634


In [12]:
results.to_csv('results.csv')

# EOF